In [8]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from skimage.transform import resize
from skimage.color import rgb2gray
from PIL import Image, ImageOps
from keras.models import model_from_json

In [2]:
dcm = pydicom.dcmread('test1.dcm')

In [3]:
dcm

(0008, 0016) SOP Class UID                       UI: Secondary Capture Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.6.1.4.1.11129.5.5.110503645592756492463169821050252582267888
(0008, 0060) Modality                            CS: 'DX'
(0008, 1030) Study Description                   LO: 'No Finding'
(0010, 0020) Patient ID                          LO: '2'
(0010, 0040) Patient's Sex                       CS: 'M'
(0010, 1010) Patient's Age                       AS: '81'
(0018, 0015) Body Part Examined                  CS: 'CHEST'
(0018, 5100) Patient Position                    CS: 'PA'
(0020, 000d) Study Instance UID                  UI: 1.3.6.1.4.1.11129.5.5.112507010803284478207522016832191866964708
(0020, 000e) Series Instance UID                 UI: 1.3.6.1.4.1.11129.5.5.112630850362182468372440828755218293352329
(0028, 0002) Samples per Pixel                   US: 1
(0028, 0004) Photometric Interpretation          CS: 'MONOCHROME2'
(0028, 0010) Rows       

In [4]:
print("Bodypart in Acquisition: %s" % (dcm.BodyPartExamined))
print('Patient Postion in Acquisition: %s' %(dcm.PatientPosition))
print('Image Modality: %s' % (dcm.Modality))

Bodypart in Acquisition: CHEST
Patient Postion in Acquisition: PA
Image Modality: DX


In [17]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)
    try:
        assert(ds.BodyPartExamined == 'CHEST')
        assert(ds.Modality == 'DX')
        assert(ds.PatientPosition in ['PA', 'AP'])
    except Exception as e:
        print('DICOM File %s contains invalid data' %(filename))
        return None
    img = ds.pixel_array
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    # todo
    
    #proc_img = rgb2gray(img)
    proc_img = resize(img, img_size)
    proc_img = (proc_img - img_mean)/img_std
    proc_img = np.resize(proc_img,(1,224,224,3))
    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # todo
    
    with open(model_path, 'r') as file:
        model_json = file.read()
        model = model_from_json(model_json)
        model.load_weights(weight_path)
        print('Model successfully loaded...')
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    # todo    
    prediction = model.predict(img) > thresh
    return prediction[0,0] 

In [18]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json'
weight_path = 'xray_class_my_model.best.hdf5'

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = 0.0
img_std = 255

my_model = load_model(model_path, weight_path)
thresh = 0.07 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
    
    img_mean = np.mean(img)
    img_std = np.std(img)
    
    
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Model successfully loaded...
Load file test1.dcm ...
True
Load file test2.dcm ...
True
Load file test3.dcm ...
True
Load file test4.dcm ...
DICOM File test4.dcm contains invalid data
Load file test5.dcm ...
DICOM File test5.dcm contains invalid data
Load file test6.dcm ...
DICOM File test6.dcm contains invalid data
